In [1]:
from threading import Thread, Lock

lock = Lock()
def Print(*args):
    with lock:
        print(*args)

Print("hi")

hi


In [2]:
from kafka import KafkaAdminClient, KafkaProducer, KafkaConsumer

# Admin

In [3]:
broker = "localhost:9092"
admin = KafkaAdminClient(bootstrap_servers=[broker])

In [4]:
admin.list_topics()

[]

In [5]:
from kafka.admin import NewTopic
from kafka.errors import TopicAlreadyExistsError

In [6]:
# admin.delete_topics(["even_nums", "odd_nums"])

In [7]:
try:
    admin.create_topics([NewTopic("even_nums", num_partitions=1, replication_factor=1)])
except TopicAlreadyExistsError:
    print("already exists")

In [8]:
try:
    admin.create_topics([NewTopic("odd_nums", num_partitions=2, replication_factor=1)])
except TopicAlreadyExistsError:
    print("already exists")

In [9]:
admin.list_topics()

['even_nums', 'odd_nums']

# Producer

In [13]:
producer = KafkaProducer(bootstrap_servers=[broker])

In [11]:
# result = producer.send("even_nums", bytes(str(0), "utf-8"))

In [12]:
# result.get()

In [14]:
import time, threading

def num_producer(topic, start, step):
    producer = KafkaProducer(bootstrap_servers=[broker])
    num = start
    while True:
        if num < 10:
            Print("send", num, "to", topic)
        producer.send(topic, bytes(str(num), "utf-8"))
        num += step
        time.sleep(1)

threading.Thread(target=num_producer, args=("even_nums", 0, 2)).start()
threading.Thread(target=num_producer, args=("odd_nums", 1, 2)).start()

send 0 to even_nums
send 1 to odd_nums
send 2 to even_nums
send 3 to odd_nums
send 5 to odd_nums
send 4 to even_nums
send 6 to even_nums
send 7 to odd_nums
send 9 to odd_nums
send 8 to even_nums


# Consumer

In [15]:
consumer = KafkaConsumer(bootstrap_servers=[broker])
batch = consumer.poll(1000)
batch

{}

In [16]:
consumer.assignment()

set()

## Manual Assignment

In [18]:
from kafka import TopicPartition

In [22]:
consumer.assign([TopicPartition("even_nums", 0)])
consumer.assignment()

{TopicPartition(topic='even_nums', partition=0)}

In [23]:
consumer.seek_to_beginning()

In [32]:
batch = consumer.poll(1000)
for topic_partition, messages in batch.items():
    for msg in messages:
        print(int(str(msg.value, "utf-8")))

200
202
204


## Automatic Assignment

In [37]:
consumer = KafkaConsumer(bootstrap_servers=[broker])
consumer.subscribe(["even_nums"])
_ = consumer.poll(1000)
print(consumer.assignment())
consumer.seek_to_beginning()

{TopicPartition(topic='even_nums', partition=0)}


In [39]:
batch = consumer.poll(1000)
for topic_partition, messages in batch.items():
    for msg in messages:
        print(int(str(msg.value, "utf-8")))

418
420
422


# Multiple Assignment

In [40]:
consumer = KafkaConsumer(bootstrap_servers=[broker])
consumer.subscribe(["even_nums", "odd_nums"])
_ = consumer.poll(1000)
print(consumer.assignment())
consumer.seek_to_beginning()

{TopicPartition(topic='odd_nums', partition=1), TopicPartition(topic='odd_nums', partition=0), TopicPartition(topic='even_nums', partition=0)}


In [56]:
batch = consumer.poll(1000)
for topic_partition, messages in batch.items():
    print(topic_partition)
    for msg in messages:
        print(int(str(msg.value, "utf-8")))

TopicPartition(topic='odd_nums', partition=0)
1255
1271
1273
1275
1277
1281
1283
1287
1293
1297
1301
1315
TopicPartition(topic='odd_nums', partition=1)
1253
1257
1259
1261
1263
1265
1267
1269
1279
1285
1289
1291
1295
1299
1303
1305
1307
1309
1311
1313
TopicPartition(topic='even_nums', partition=0)
1252
1254
1256
1258
1260
1262
1264
1266
1268
1270
1272
1274
1276
1278
1280
1282
1284
1286
1288
1290
1292
1294
1296
1298
1300
1302
1304
1306
1308
1310
1312
1314


In [57]:
positions = {}
for tp in consumer.assignment():
    pos = consumer.position(tp)
    positions[tp] = pos
positions

{TopicPartition(topic='odd_nums', partition=1): 328,
 TopicPartition(topic='odd_nums', partition=0): 330,
 TopicPartition(topic='even_nums', partition=0): 658}

In [58]:
consumer2 = KafkaConsumer(bootstrap_servers=[broker])

In [59]:
consumer2.assign(positions.keys())
for tp, position in positions.items():
    consumer2.seek(tp, position)

In [60]:
batch = consumer2.poll(1000)
for topic_partition, messages in batch.items():
    print(topic_partition)
    for msg in messages:
        print(int(str(msg.value, "utf-8")))

TopicPartition(topic='odd_nums', partition=1)
1317
1319
1323
1325
1327
1329
1333
1337
1339
1341
1343
1355
1359
1369
1371
TopicPartition(topic='odd_nums', partition=0)
1321
1331
1335
1345
1347
1349
1351
1353
1357
1361
1363
1365
1367
1373
TopicPartition(topic='even_nums', partition=0)
1316
1318
1320
1322
1324
1326
1328
1330
1332
1334
1336
1338
1340
1342
1344
1346
1348
1350
1352
1354
1356
1358
1360
1362
1364
1366
1368
1370
1372


# Consumer Groups (2 of them reading even numbers)

In [64]:
def consume_odds(group, thread):
    consumer = KafkaConsumer(bootstrap_servers=[broker], group_id=group)
    consumer.subscribe(["odd_nums"])
    for i in range(10):
        batch = consumer.poll(1000)
        for tp, messages in batch.items():
            for msg in messages:
                Print(group, thread, msg.value)

threading.Thread(target=consume_odds, args=("g1","t1")).start()
threading.Thread(target=consume_odds, args=("g2","t2")).start()
threading.Thread(target=consume_odds, args=("g2","t3")).start()

g1 t1 b'2025'
g2 t2 b'2025'
g1 t1 b'2027'
g2 t2 b'2027'
g2 t2 b'2029'
g1 t1 b'2029'
g1 t1 b'2031'
g2 t3 b'2031'
g2 t3 b'2033'
g1 t1 b'2033'
g2 t3 b'2035'
g1 t1 b'2035'
g1 t1 b'2037'
g2 t2 b'2037'
g1 t1 b'2039'
g2 t3 b'2039'
g1 t1 b'2041'
g2 t2 b'2041'
g2 t2 b'2043'
g1 t1 b'2043'
